<a href="https://colab.research.google.com/github/rajeshmore1/Deep-Learning-Mentorship--Rajesh-More/blob/main/Keras_Tuner_Decide_Number_of_Hidden_Layers_And_Neuron_In_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keras Tuner- Decide Number of Hidden Layers And Neuron In Neural Network


In [3]:
pip install keras-tuner --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 4.3 MB/s 
     |████████████████████████████████| 1.6 MB 36.0 MB/s 


In [6]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch

In [7]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Real_Combine.csv")

In [8]:
df.head()


,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [9]:
X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

# Hyperparameters
1. How many number of hidden layers we should have?
2. How many number of neurons we should have in hidden layers?
3. Learning Rate

In [10]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [11]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')

In [12]:
tuner.search_space_summary()


Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [14]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 12s]
val_mean_absolute_error: nan

Best val_mean_absolute_error So Far: nan
Total elapsed time: 00h 01m 10s


# Other approach

To demonstrate hyperparameter tuning methods, we’ll use keras tuner library to tune a regression model on the Boston housing price dataset. This dataset contains 13 attributes with 404 and 102 training and testing samples respectively. We’ll use tensorflow as keras backend so make sure you have tensorflow installed on your machines. I’m using tensorflow version ‘2.1.0’ and kerastuner version ‘1.0.1’. Tensorflow 2.0.x comes up with keras so you don’t need to install keras separately if you have version 2.0.x. You can check the version you have using the code below:

In [15]:
import tensorflow as tf
import kerastuner as kt
print(tf.__version__)
print(kt.__version__)

2.9.2
1.0.3


# Load the dataset
Boston housing price regression dataset can be downloaded directly using keras. 

In [16]:
from tensorflow.keras.datasets import boston_housing
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

57026/57026 [==============================] - 0s 0us/step


This is the regression model I’ll use in this demo. The code below shows how the model was built without any tuning.

In [17]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import models, layers
# set random seed
from numpy.random import seed
seed(42)
import tensorflow
tensorflow.random.set_seed(42)
# preprocessing - normalization
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)
# model building
model = models.Sequential()
model.add(layers.Dense(8, activation='relu', input_shape=(x_train.shape[1],)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1))
# compile model using rmsprop
model.compile(optimizer='rmsprop',loss='mse',metrics=['mse'])
# model training
history = model.fit(x_train_scaled, y_train, validation_split=0.2, epochs=10)
# model evaluation
model.evaluate(x_test_scaled, y_test)

Epoch 1/10
11/11 [==============================] - 1s 29ms/step - loss: 568.7303 - mse: 568.7303 - val_loss: 627.1449 - val_mse: 627.1449
Epoch 2/10
11/11 [==============================] - 0s 6ms/step - loss: 555.4802 - mse: 555.4802 - val_loss: 614.4634 - val_mse: 614.4634
Epoch 3/10
11/11 [==============================] - 0s 6ms/step - loss: 542.7706 - mse: 542.7706 - val_loss: 603.5865 - val_mse: 603.5865
Epoch 4/10
11/11 [==============================] - 0s 6ms/step - loss: 532.2389 - mse: 532.2389 - val_loss: 592.0689 - val_mse: 592.0689
Epoch 5/10
11/11 [==============================] - 0s 4ms/step - loss: 524.3663 - mse: 524.3663 - val_loss: 580.4174 - val_mse: 580.4174
Epoch 6/10
11/11 [==============================] - 0s 5ms/step - loss: 510.7414 - mse: 510.7414 - val_loss: 567.9651 - val_mse: 567.9651
Epoch 7/10
11/11 [==============================] - 0s 6ms/step - loss: 498.7845 - mse: 498.7845 - val_loss: 554.9809 - val_mse: 554.9809
Epoch 8/10
11/11 [===============

[488.33251953125, 488.33251953125]

This model has a MSE of around 434. I have set the random seed in numpy and tensorflow to 42 to get reproducible results. Despite doing so, I still get slightly different results every time I run the code. Let me know in the comments what else I missed make this reproducible.

# Tuning with Keras Tuner
To start tuning the model in keras tuner, let’s define a hypermodel first. Hypermodel is a keras tuner class that lets you define the model with a searchable space and build it.

Create a class that inherits from kerastuner.HyperModel, like so:

In [25]:
from kerastuner import HyperModel
class RegressionHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape
    def build(self, hp):
        model = Sequential()
        model.add(
            layers.Dense(
                units=hp.Int('units', 8, 64, 4, default=8),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                input_shape=input_shape
            )
        )
        
        model.add(
            layers.Dense(
                units=hp.Int('units', 16, 64, 4, default=16),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu')
            )
        )
        
        model.add(
            layers.Dropout(
                hp.Float(
                    'dropout',
                    min_value=0.0,
                    max_value=0.1,
                    default=0.005,
                    step=0.01)
            )
        )
        
        model.add(layers.Dense(1))
        
        model.compile(
            optimizer='rmsprop',loss='mse',metrics=['mse']
        )
        
        return model

This is the same model we built earlier, except that for every hyperparameter, we defined a search space. You may have noticed hp.Int, hp.Float, and hp.Choice, these are used to define a search space for a hyperparameter that accepts an integer, float and a category respectively. ‘hp’ is an alias for Keras Tuner’s HyperParameters class.

Hyperparameter such as the number of units in a dense layer accepts an integer, hence, hp.Int is used to define a range of integers to try. Similarly, the dropout rate accepts a float value so hp.Float is used. Both hp.Int and hp.Float requires a name, minimum value and maximum value, while the step size and default value is optional.

The hp.Int search space below is named, “units”, and will have values from 8 to 64 in multiples of 4, and a default value of 8. hp. Float is used similarly as hp.Int but accepts float values.

In [26]:
hp.Int('units', 8, 64, 4, default=8)


hp.Choice is used to define a categorical hyperparameter such as the activation function. The search space below, named “dense_activation”, will choose between “relu”, “tanh”, and “sigmoid” functions, with a default value set to “relu”.

In [ ]:
hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid'], default='relu')

# Instantiate HyperModel
Let’s instantiate a hypermodel object. Input shape varies per dataset and the problem you are trying to solve.

In [27]:
input_shape = (x_train.shape[1],)
hypermodel = RegressionHyperModel(input_shape)

# Random Search
As the name suggests, this hyperparameter tuning method randomly tries a combination of hyperparameters from a given search space. To use this method in keras tuner, let’s define a tuner using one of the available Tuners.

In [29]:
from keras.models import Sequential

tuner_rs = RandomSearch(
            hypermodel,
            objective='mse',
            seed=42,
            max_trials=10,
            executions_per_trial=2)

Run the random search tuner using the search method.



In [30]:
tuner_rs.search(x_train_scaled, y_train, epochs=10, validation_split=0.2, verbose=0)

Select the best combination of hyperparameters the tuner had tried and evaluate.

In [31]:
best_model = tuner_rs.get_best_models(num_models=1)[0]
loss, mse = best_model.evaluate(x_test_scaled, y_test)

4/4 [==============================] - 0s 3ms/step - loss: 55.0461 - mse: 55.0461


Random search’s MSE is 53.48, a very big improvement from not performing any tuning at all.